In [33]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

In [34]:
gebruikers_df = pd.read_csv(r'..\..\data\gebruikersinformatie.csv', delimiter = ';')
evenement_df = pd.read_csv(r'..\..\data\evenementinformatie.csv', delimiter = ';')
aanwezigheid_df = pd.read_csv(r'..\..\data\evenementaanwezigheid.csv', delimiter = ';')
bericht_df = pd.read_csv(r'..\..\data\berichtinteracties_met_sentiment.csv', delimiter = ';')

In [35]:
# Merge bericht_df with evenement_df
bericht_df = bericht_df.merge(evenement_df, on = "EvenementID", how = "left")

# Merge the result with gebruikers_df
bericht_df = bericht_df.merge(gebruikers_df, on = "GebruikerID", how = "left")

# Rename columns for consistency
bericht_df.rename(
    columns = {col: f"Evenement_{col}" for col in evenement_df.columns if col != "EvenementID"},
    inplace = True
)

bericht_df.rename(
    columns = {col: f"Gebruiker_{col}" for col in gebruikers_df.columns if col != "GebruikerID"},
    inplace = True
)

bericht_df

,BerichtID,BerichtDatum,GebruikerID,EvenementID,BerichtSentiment,OntvangenLikes,OntvangenReacties,Evenement_EvenementType,Evenement_EvenementDatum,Evenement_OrganisatorID,Gebruiker_RegistratieDatum,Gebruiker_LaatsteLogin,Gebruiker_Leeftijd,Gebruiker_Studierichting,Gebruiker_Studiejaar,Gebruiker_Lidmaatschapstype
0,B001,18-5-2023,G309,E012,Neutraal,5,8,Hackathon,24-4-2023,G271,23-12-2023,2-8-2024,19.0,Security & Cloud,Derdejaars,Premium
1,B002,6-10-2023,G021,E039,Positief,9,6,Workshop,1-9-2023,G199,14-1-2023,3-2-2023,17.0,Human computer interaction,Eerstejaars,Premium
2,B003,7-7-2023,G111,E001,Positief,8,5,BBQ,24-2-2023,G296,19-4-2023,10-6-2023,17.0,Human computer interaction,Derdejaars,Premium
3,B004,5-10-2023,G168,E015,Negatief,4,0,Hackathon,20-7-2023,G199,24-2-2023,18-3-2023,19.0,Human computer interaction,Tweedejaars,Premium
4,B005,22-1-2023,G076,E017,Neutraal,1,8,Lezing,14-12-2023,G201,25-3-2023,30-11-2023,17.0,Data engineering,Tweedejaars,Premium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,B496,16-2-2023,G263,E010,Neutraal,19,0,BBQ,19-12-2023,G135,28-8-2023,20-5-2024,12.0,Security & Cloud,Vierdejaars,Premium
496,B497,22-3-2023,G011,E004,Positief,17,6,BBQ,13-5-2023,G096,14-2-2023,27-3-2023,17.0,Human computer interaction,Eerstejaars,Premium
497,B498,18-12-2023,G005,E004,Positief,0,4,BBQ,13-5-2023,G096,6-5-2023,5-1-2024,17.0,Human computer interaction,Eerstejaars,Premium
498,B499,5-6-2023,G243,E013,Positief,20,9,Lezing,17-8-2023,G161,23-5-2023,3-3-2024,22.0,Human computer interaction,Vierdejaars,Premium


In [36]:
# Remove nonvalid entries
bericht_df = bericht_df[~(bericht_df['Gebruiker_Leeftijd'].isna())]
bericht_df = bericht_df[~(bericht_df['Gebruiker_Lidmaatschapstype'] == 'x')]
bericht_df

,BerichtID,BerichtDatum,GebruikerID,EvenementID,BerichtSentiment,OntvangenLikes,OntvangenReacties,Evenement_EvenementType,Evenement_EvenementDatum,Evenement_OrganisatorID,Gebruiker_RegistratieDatum,Gebruiker_LaatsteLogin,Gebruiker_Leeftijd,Gebruiker_Studierichting,Gebruiker_Studiejaar,Gebruiker_Lidmaatschapstype
0,B001,18-5-2023,G309,E012,Neutraal,5,8,Hackathon,24-4-2023,G271,23-12-2023,2-8-2024,19.0,Security & Cloud,Derdejaars,Premium
1,B002,6-10-2023,G021,E039,Positief,9,6,Workshop,1-9-2023,G199,14-1-2023,3-2-2023,17.0,Human computer interaction,Eerstejaars,Premium
2,B003,7-7-2023,G111,E001,Positief,8,5,BBQ,24-2-2023,G296,19-4-2023,10-6-2023,17.0,Human computer interaction,Derdejaars,Premium
3,B004,5-10-2023,G168,E015,Negatief,4,0,Hackathon,20-7-2023,G199,24-2-2023,18-3-2023,19.0,Human computer interaction,Tweedejaars,Premium
4,B005,22-1-2023,G076,E017,Neutraal,1,8,Lezing,14-12-2023,G201,25-3-2023,30-11-2023,17.0,Data engineering,Tweedejaars,Premium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,B496,16-2-2023,G263,E010,Neutraal,19,0,BBQ,19-12-2023,G135,28-8-2023,20-5-2024,12.0,Security & Cloud,Vierdejaars,Premium
496,B497,22-3-2023,G011,E004,Positief,17,6,BBQ,13-5-2023,G096,14-2-2023,27-3-2023,17.0,Human computer interaction,Eerstejaars,Premium
497,B498,18-12-2023,G005,E004,Positief,0,4,BBQ,13-5-2023,G096,6-5-2023,5-1-2024,17.0,Human computer interaction,Eerstejaars,Premium
498,B499,5-6-2023,G243,E013,Positief,20,9,Lezing,17-8-2023,G161,23-5-2023,3-3-2024,22.0,Human computer interaction,Vierdejaars,Premium


In [37]:
# Convert ID to ints
to_convert = ["BerichtID", "GebruikerID", "EvenementID", "Evenement_OrganisatorID"]

for col in to_convert:
    bericht_df[col] = bericht_df[col].str[1:].astype(int)
    
bericht_df

,BerichtID,BerichtDatum,GebruikerID,EvenementID,BerichtSentiment,OntvangenLikes,OntvangenReacties,Evenement_EvenementType,Evenement_EvenementDatum,Evenement_OrganisatorID,Gebruiker_RegistratieDatum,Gebruiker_LaatsteLogin,Gebruiker_Leeftijd,Gebruiker_Studierichting,Gebruiker_Studiejaar,Gebruiker_Lidmaatschapstype
0,1,18-5-2023,309,12,Neutraal,5,8,Hackathon,24-4-2023,271,23-12-2023,2-8-2024,19.0,Security & Cloud,Derdejaars,Premium
1,2,6-10-2023,21,39,Positief,9,6,Workshop,1-9-2023,199,14-1-2023,3-2-2023,17.0,Human computer interaction,Eerstejaars,Premium
2,3,7-7-2023,111,1,Positief,8,5,BBQ,24-2-2023,296,19-4-2023,10-6-2023,17.0,Human computer interaction,Derdejaars,Premium
3,4,5-10-2023,168,15,Negatief,4,0,Hackathon,20-7-2023,199,24-2-2023,18-3-2023,19.0,Human computer interaction,Tweedejaars,Premium
4,5,22-1-2023,76,17,Neutraal,1,8,Lezing,14-12-2023,201,25-3-2023,30-11-2023,17.0,Data engineering,Tweedejaars,Premium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,496,16-2-2023,263,10,Neutraal,19,0,BBQ,19-12-2023,135,28-8-2023,20-5-2024,12.0,Security & Cloud,Vierdejaars,Premium
496,497,22-3-2023,11,4,Positief,17,6,BBQ,13-5-2023,96,14-2-2023,27-3-2023,17.0,Human computer interaction,Eerstejaars,Premium
497,498,18-12-2023,5,4,Positief,0,4,BBQ,13-5-2023,96,6-5-2023,5-1-2024,17.0,Human computer interaction,Eerstejaars,Premium
498,499,5-6-2023,243,13,Positief,20,9,Lezing,17-8-2023,161,23-5-2023,3-3-2024,22.0,Human computer interaction,Vierdejaars,Premium


In [38]:
# Convert columns to datetime datatype and extract month
date_columns = ['BerichtDatum', 'Evenement_EvenementDatum', 'Gebruiker_RegistratieDatum', 'Gebruiker_LaatsteLogin']

for col in date_columns:
    bericht_df[col] = pd.to_datetime(bericht_df[col], dayfirst = True)

for col in date_columns:
    bericht_df[f'{col}_month'] = bericht_df[col].dt.month

bericht_df

,BerichtID,BerichtDatum,GebruikerID,EvenementID,BerichtSentiment,OntvangenLikes,OntvangenReacties,Evenement_EvenementType,Evenement_EvenementDatum,Evenement_OrganisatorID,Gebruiker_RegistratieDatum,Gebruiker_LaatsteLogin,Gebruiker_Leeftijd,Gebruiker_Studierichting,Gebruiker_Studiejaar,Gebruiker_Lidmaatschapstype,BerichtDatum_month,Evenement_EvenementDatum_month,Gebruiker_RegistratieDatum_month,Gebruiker_LaatsteLogin_month
0,1,2023-05-18,309,12,Neutraal,5,8,Hackathon,2023-04-24,271,2023-12-23,2024-08-02,19.0,Security & Cloud,Derdejaars,Premium,5,4,12,8
1,2,2023-10-06,21,39,Positief,9,6,Workshop,2023-09-01,199,2023-01-14,2023-02-03,17.0,Human computer interaction,Eerstejaars,Premium,10,9,1,2
2,3,2023-07-07,111,1,Positief,8,5,BBQ,2023-02-24,296,2023-04-19,2023-06-10,17.0,Human computer interaction,Derdejaars,Premium,7,2,4,6
3,4,2023-10-05,168,15,Negatief,4,0,Hackathon,2023-07-20,199,2023-02-24,2023-03-18,19.0,Human computer interaction,Tweedejaars,Premium,10,7,2,3
4,5,2023-01-22,76,17,Neutraal,1,8,Lezing,2023-12-14,201,2023-03-25,2023-11-30,17.0,Data engineering,Tweedejaars,Premium,1,12,3,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,496,2023-02-16,263,10,Neutraal,19,0,BBQ,2023-12-19,135,2023-08-28,2024-05-20,12.0,Security & Cloud,Vierdejaars,Premium,2,12,8,5
496,497,2023-03-22,11,4,Positief,17,6,BBQ,2023-05-13,96,2023-02-14,2023-03-27,17.0,Human computer interaction,Eerstejaars,Premium,3,5,2,3
497,498,2023-12-18,5,4,Positief,0,4,BBQ,2023-05-13,96,2023-05-06,2024-01-05,17.0,Human computer interaction,Eerstejaars,Premium,12,5,5,1
498,499,2023-06-05,243,13,Positief,20,9,Lezing,2023-08-17,161,2023-05-23,2024-03-03,22.0,Human computer interaction,Vierdejaars,Premium,6,8,5,3


In [39]:
# Create boolean dataframe
dummies_df = pd.DataFrame()

# Bericht columns
dummies_df["BerichtDatum_is_weekday"] = bericht_df["BerichtDatum"].dt.weekday < 5
dummies_df["BerichtDatum_is_weekend"] = bericht_df["BerichtDatum"].dt.weekday >= 5

dummies_df = pd.concat([dummies_df, pd.get_dummies(bericht_df['BerichtSentiment'], prefix = 'BerichtSentiment')], axis = 1)

dummies_df[f'Low likes'] = bericht_df["OntvangenLikes"] < bericht_df["OntvangenLikes"].mean()
dummies_df[f'High likes'] = bericht_df["OntvangenLikes"] >= bericht_df["OntvangenLikes"].mean()

dummies_df[f'Low reacties'] = bericht_df["OntvangenReacties"] < bericht_df["OntvangenReacties"].mean()
dummies_df[f'High reacties'] = bericht_df["OntvangenReacties"] >= bericht_df["OntvangenReacties"].mean()


# Evenement columns
dummies_df = pd.concat([dummies_df, pd.get_dummies(bericht_df['Evenement_EvenementType'], prefix = 'EvenementType')], axis = 1)

dummies_df["EvenementDatum_is_weekday"] = bericht_df["Evenement_EvenementDatum"].dt.weekday < 5
dummies_df["EvenementDatum_is_weekend"] = bericht_df["Evenement_EvenementDatum"].dt.weekday >= 5

dummies_df = pd.concat([dummies_df, pd.get_dummies(bericht_df['Evenement_EvenementDatum_month'], prefix = 'EvenementDatum_month')], axis = 1)


# Gebruiker columns
dummies_df[f'Younger'] = bericht_df["Gebruiker_Leeftijd"] <= bericht_df["Gebruiker_Leeftijd"].mode()[0]
dummies_df[f'Older'] = bericht_df["Gebruiker_Leeftijd"] > bericht_df["Gebruiker_Leeftijd"].mode()[0]

dummies_df = pd.concat([dummies_df, pd.get_dummies(bericht_df['Gebruiker_Studierichting'].replace('Data engineering', 'Data Engineering'), prefix = 'Studierichting')], axis = 1)
dummies_df = pd.concat([dummies_df, pd.get_dummies(bericht_df['Gebruiker_Studiejaar'], prefix = 'Studiejaar')], axis = 1)
dummies_df = pd.concat([dummies_df, pd.get_dummies(bericht_df['Gebruiker_Lidmaatschapstype'], prefix = 'Lidmaatschapstype')], axis = 1)

dummies_df

,BerichtDatum_is_weekday,BerichtDatum_is_weekend,BerichtSentiment_Negatief,BerichtSentiment_Neutraal,BerichtSentiment_Positief,Low likes,High likes,Low reacties,High reacties,EvenementType_BBQ,...,Studierichting_Data Engineering,Studierichting_Human computer interaction,Studierichting_Security & Cloud,Studierichting_Software engineering,Studiejaar_Derdejaars,Studiejaar_Eerstejaars,Studiejaar_Tweedejaars,Studiejaar_Vierdejaars,Lidmaatschapstype_Premium,Lidmaatschapstype_Regulier
0,True,False,False,True,False,True,False,False,True,False,...,False,False,True,False,True,False,False,False,True,False
1,True,False,False,False,True,True,False,False,True,False,...,False,True,False,False,False,True,False,False,True,False
2,True,False,False,False,True,True,False,True,False,True,...,False,True,False,False,True,False,False,False,True,False
3,True,False,True,False,False,True,False,True,False,False,...,False,True,False,False,False,False,True,False,True,False
4,False,True,False,True,False,True,False,False,True,False,...,True,False,False,False,False,False,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,True,False,False,True,False,False,True,True,False,True,...,False,False,True,False,False,False,False,True,True,False
496,True,False,False,False,True,False,True,False,True,True,...,False,True,False,False,False,True,False,False,True,False
497,True,False,False,False,True,True,False,True,False,True,...,False,True,False,False,False,True,False,False,True,False
498,True,False,False,False,True,False,True,False,True,False,...,False,True,False,False,False,False,False,True,True,False


In [40]:
# Create frequent itemsets and association rules
freq_itemsets = apriori(dummies_df, min_support = 0.1, use_colnames = True)
association_rules(freq_itemsets, metric = 'lift', min_threshold = 1.25)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(BerichtDatum_is_weekend),(Studierichting_Data Engineering),0.306748,0.319018,0.126789,0.413333,1.295641,0.028931,1.160764,0.329146
1,(Studierichting_Data Engineering),(BerichtDatum_is_weekend),0.319018,0.306748,0.126789,0.397436,1.295641,0.028931,1.150503,0.335077
2,(BerichtDatum_is_weekend),(Studiejaar_Tweedejaars),0.306748,0.274029,0.106339,0.346667,1.265075,0.022282,1.111181,0.302246
3,(Studiejaar_Tweedejaars),(BerichtDatum_is_weekend),0.274029,0.306748,0.106339,0.388060,1.265075,0.022282,1.132874,0.288624
4,(EvenementDatum_is_weekend),(EvenementType_BBQ),0.251534,0.325153,0.145194,0.577236,1.775272,0.063407,1.596272,0.583468
...,...,...,...,...,...,...,...,...,...,...
163,"(Older, EvenementType_Lezing)","(Low reacties, EvenementDatum_is_weekday)",0.224949,0.390593,0.116564,0.518182,1.326654,0.028701,1.264807,0.317687
164,"(Lidmaatschapstype_Premium, Older, EvenementTy...",(EvenementDatum_is_weekday),0.118609,0.748466,0.112474,0.948276,1.266959,0.023699,4.862986,0.239063
165,"(Lidmaatschapstype_Premium, EvenementDatum_is_...","(Older, EvenementType_Lezing)",0.396728,0.224949,0.112474,0.283505,1.260309,0.023231,1.081726,0.342373
166,"(Older, EvenementType_Lezing)","(Lidmaatschapstype_Premium, EvenementDatum_is_...",0.224949,0.396728,0.112474,0.500000,1.260309,0.023231,1.206544,0.266491
